# Deep, deep cleaning. Data Dictionary, calculated columns, useless crowd, and more

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings("ignore")

import platform
print('Versions:')
print('  python', platform.python_version())
n = ('numpy', 'pandas')
nn = (np, pd)
for a, b in zip(n, nn):
    print('  --', str(a), b.__version__)

Versions:
  python 3.6.8
  -- numpy 1.17.3
  -- pandas 0.25.2


In [2]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('colheader_justify', 'left', 'display.max_colwidth', -1, )

In [3]:
def dicts(w):
    """Print data and value dictionaries for columns which name contains specified text"""
    print('Data Dictionary:')
    display(HTML(ddict[ddict.index.str.contains(w)].to_html()))
    print('Value Description:')
    tdf = ddes[ddes['Feature'].str.contains(w)]
    tdf = tdf[['Feature', 'Val', 'Value']]
    display(HTML(tdf.to_html()))

In [4]:
def noinfo(df, threshold):
    """Return a list, and number, of flat columns:
    1. If missing values more than a threshold
    2. If top value is used more than threshold times"""
    flat = []
    for col in df.columns:
        if df[col].value_counts(normalize=True).to_frame().T.iloc[0,0] > threshold:
            flat.append(col)  
        if (df[col].isnull().sum() * 100 / len(df.index)) > (threshold*100):
            flat.append(col)
    flat = pd.unique(flat).tolist()
    print(len(flat), 'columns are flat') 
    return flat

In [5]:
train = pd.read_csv('.../HPriceKaggleTrain.csv', index_col='Id')
print('train shape:', train.shape)
test = pd.read_csv('.../HPriceKaggleTest.csv', index_col='Id')
print('test shape:', test.shape)
df = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
print('Concatenated data set has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

train shape: (1460, 80)
test shape: (1459, 79)
Concatenated data set has 2919 rows and 80 columns


The importance of data cleaning is hard to overestimate - incorrect or inconsistent data leads to false conclusions.  

And so, we should understand the data as much deep, as it possible.  

Creating a Data Dictionary helps here a lot.

In [6]:
ddict = pd.read_excel('.../HPricesDDict.xlsx', index_col=0)
ddict.sample()

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
TotRmsAbvGrd,Total rooms above grade (does not include bathrooms),14,0.00,6,28.91,int64,discrete,quantity


Data Dictionary has following columns:
- 'Description' - feature meaning
- '#Unique' - number of unique values in the columns where NaN calculated as value also  
- '%Missing' - % of missing values
- 'TopValue' - the most used value  
- '%UsedTop' - % of using top value  
- 'Dtype' - column's python data type 
- 'Type' - column's data type 
- 'Unit' - given unit of value 

In [7]:
ddes = pd.read_excel('.../HPricesDDescribtion.xlsx', index_col=0)
ddes.sample()

,Feature,Descript,Val,Value,Type,Unit
114,OverallQual,Rates the overall material and finish of the house,3,Fair,ordinal,rate


A Data Description file was created first and contains fewer common information, but it has description for every categorical value.
- 'Feature' - main data set column name
- 'Descript' - feature meaning
- 'Val' - value
- 'Value' - value meaning
- 'Type' - column's data type
- 'Unit' - given unit of value

# Memory usage

In [8]:
mu_before = round(df.memory_usage(index=True).sum()/1024, 2)
print(mu_before, 'MB')

1824.5 MB


In [9]:
df.memory_usage(deep=True).nlargest().to_frame()

,0
CentralAir,192654
PavedDrive,192654
RoofMatl,186811
Exterior1st,186735
Exterior2nd,186720


As we know, casting object data to categorical decrease a memory usage tremendously. Let's take a look.

In [10]:
cols = df.dtypes[df.dtypes=='object'].index.to_list()
len(cols)

43

We have 43 object features here.  
Let's find a columns with a maximum of unique values to make a decision (we can cast not very vary features only).

In [11]:
df[cols].describe(include='all').loc['unique', :].sort_values(ascending=False).head()

Neighborhood    25
Exterior2nd     16
Exterior1st     15
Condition1      9 
SaleType        9 
Name: unique, dtype: object

Thus, we can change data types for all this features and check the memory usage again

In [12]:
for col in cols:
    df[col] = df[col].astype('category')
    
print(df.memory_usage(index=True).sum()/1024, 'MB')    

976.8798828125 MB


# Duplicates
Simple but important step

In [13]:
df.duplicated().sum()

0

# Feature Selection

### Extraction of calculated columns:

In [14]:
sf = ddict[ddict['Unit']=='square feet']
sf

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
1stFlrSF,First Floor square feet,1083,0.00,864,1.58,int64,cont,square feet
2ndFlrSF,Second floor square feet,635,0.00,0,57.14,int64,cont,square feet
3SsnPorch,Three season porch area in square feet,31,0.00,0,98.73,int64,cont,square feet
BsmtFinSF1,Type 1 finished square feet,992,0.03,0,31.83,float64,cont,square feet
BsmtFinSF2,Type 2 finished square feet,273,0.03,0,88.08,float64,cont,square feet
BsmtUnfSF,Unfinished square feet of basement area,1136,0.03,0,8.26,float64,cont,square feet
EnclosedPorch,Enclosed porch area in square feet,183,0.00,0,84.28,int64,cont,square feet
GarageArea,Size of garage in square feet,604,0.03,0,5.38,float64,cont,square feet
GrLivArea,Above grade (ground) living area square feet,1292,0.00,864,1.40,int64,cont,square feet
LotArea,Lot size in square feet,1951,0.00,9600,1.51,int64,cont,square feet


In [15]:
df[sf.index].sample()

,1stFlrSF,2ndFlrSF,3SsnPorch,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,EnclosedPorch,GarageArea,GrLivArea,LotArea,LowQualFinSF,MasVnrArea,OpenPorchSF,PoolArea,ScreenPorch,TotalBsmtSF,WoodDeckSF
2681,1358,1368,0,527.00,0.00,815.00,168,725.00,2726,10019,0,397.00,169,0,0,"1,342.00",307


In [16]:
df[['1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea']].sample()

,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea
633,1056,0,0,1056


Looks like there are calculated columns here, let's check

In [17]:
df['Temp'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['LowQualFinSF']
df['Temp'].equals(df['GrLivArea'])

True

In [18]:
df[['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']].sample()

,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
1852,552.00,393.00,104.00,"1,049.00"


In [19]:
df['Temp'] = df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['BsmtUnfSF']
df['Temp'].equals(df['TotalBsmtSF'])

True

In [20]:
df[['3SsnPorch', 'EnclosedPorch', 'OpenPorchSF', 'ScreenPorch', 'GarageArea', 'LotArea', 'MasVnrArea',  'PoolArea', 'WoodDeckSF']].sample()

,3SsnPorch,EnclosedPorch,OpenPorchSF,ScreenPorch,GarageArea,LotArea,MasVnrArea,PoolArea,WoodDeckSF
2174,0,0,116,0,725.00,9720,134.00,0,168


In [21]:
q = ddict[ddict['Unit']=='quantity']
q

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
BedroomAbvGr,Bedrooms above grade (does NOT include basement bedrooms),8,0.00,3,54.68,int64,discrete,quantity
BsmtFullBath,Basement full bathrooms,5,0.07,0,58.41,float64,discrete,quantity
BsmtHalfBath,Basement half bathrooms,4,0.07,0,93.94,float64,discrete,quantity
Fireplaces,Number of fireplaces,5,0.00,0,48.65,int64,discrete,quantity
FullBath,Full bathrooms above grade,5,0.00,2,52.42,int64,discrete,quantity
GarageCars,Size of garage in car capacity,7,0.03,2,54.61,float64,discrete,quantity
HalfBath,Half baths above grade,3,0.00,0,62.83,int64,discrete,quantity
KitchenAbvGr,Kitchens above grade,4,0.00,1,95.41,int64,discrete,quantity
TotRmsAbvGrd,Total rooms above grade (does not include bathrooms),14,0.00,6,28.91,int64,discrete,quantity


In [22]:
df[q.index].sample()

,BedroomAbvGr,BsmtFullBath,BsmtHalfBath,Fireplaces,FullBath,GarageCars,HalfBath,KitchenAbvGr,TotRmsAbvGrd
1977,4,0.00,0.00,1,2,3.00,1,1,9


In [23]:
df['Temp'] = df['FullBath'] + df['HalfBath'] + df['KitchenAbvGr'] + df['BedroomAbvGr']
df['Temp'].equals(df['TotRmsAbvGrd'])

False

Now we can drop the 'Temp' column we created and discovered calculated columns:

In [24]:
df = df.drop('Temp', axis=1)
calculated = ['GrLivArea', 'TotalBsmtSF']
df = df.drop(calculated, axis=1)
df.shape

(2919, 78)

In [25]:
#do the same with an auxiliary tables
ddict = ddict.loc[~ddict.index.isin(calculated)]
ddes = ddes.loc[~ddes['Feature'].isin(calculated)]
ddict.shape

(78, 8)

### Variance Threshold

When a feature doesn’t vary much within itself, it generally has very little predictive power.  

In [26]:
dicts('Pool')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
PoolArea,Pool area in square feet,14,0.00,0,99.55,int64,cont,square feet
PoolQC,Pool quality,4,99.66,Ex,0.14,object,ordinal,rate


Value Description:


,Feature,Val,Value
317,PoolArea,NaN,NaN
321,PoolQC,Fa,Fair
320,PoolQC,TA,Average/Typical
319,PoolQC,Gd,Good
318,PoolQC,Ex,Excellent
322,PoolQC,NaN,No Pool


How to decide what threshold to use for removing low-variance features?  
I haven't find any deep tutorial about it yet. 
So, I've decided to use 80% for study cases [as here](https://scikit-learn.org/stable/modules/feature_selection.html) and try several in a real life cases.

In [27]:
flat = noinfo(df, 0.8)
# flat

33 columns are flat


In [28]:
ddict.loc[ddict.index.isin(flat)]

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
3SsnPorch,Three season porch area in square feet,31,0.00,0,98.73,int64,cont,square feet
Alley,Type of alley access to property,3,93.22,Grvl,4.11,object,nominal,-
BldgType,Type of dwelling,5,0.00,1Fam,83.08,object,nominal,-
BsmtCond,Evaluates the general condition of the basement,5,2.81,TA,89.28,object,ordinal,rate
BsmtFinSF2,Type 2 finished square feet,273,0.03,0,88.08,float64,cont,square feet
BsmtFinType2,Rating of basement finished area (if multiple types),7,2.74,Unf,85.41,object,ordinal,rate
BsmtHalfBath,Basement half bathrooms,4,0.07,0,93.94,float64,discrete,quantity
CentralAir,Central air conditioning,2,0.00,Y,93.29,object,binary,-
Condition1,Proximity to various conditions,9,0.00,Norm,86.02,object,nominal,-
Condition2,Proximity to various conditions (if more than one is pr,8,0.00,Norm,98.97,object,nominal,-


In [29]:
# deleting useless data
df = df.drop(flat, axis=1)
df.shape

(2919, 45)

In [30]:
# do the same with auxiliary
ddes = ddes.loc[~ddes['Feature'].isin(flat)].reset_index(drop=True)
ddict = ddict.loc[~ddict.index.isin(flat)]
len(ddict)

45

# Missing values

In [31]:
tdf = df.isnull().sum().sort_values(ascending=False)
tdf.loc[(tdf != 0)]

SalePrice       1459
FireplaceQu     1420
LotFrontage     486 
GarageFinish    159 
GarageYrBlt     159 
GarageType      157 
BsmtExposure    82  
BsmtQual        81  
BsmtFinType1    79  
MasVnrType      24  
MasVnrArea      23  
MSZoning        4   
BsmtFullBath    2   
Exterior1st     1   
Exterior2nd     1   
BsmtUnfSF       1   
BsmtFinSF1      1   
KitchenQual     1   
GarageCars      1   
GarageArea      1   
dtype: int64

Let's do it step by step

In [32]:
dicts('Fire')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
FireplaceQu,Fireplace quality,6,48.65,Gd,25.49,object,ordinal,rate
Fireplaces,Number of fireplaces,5,0.00,0,48.65,int64,discrete,quantity


Value Description:


,Feature,Val,Value
63,FireplaceQu,Po,Poor - Ben Franklin Stove
64,FireplaceQu,Ex,Excellent - Exceptional Masonry Fireplace
65,FireplaceQu,NaN,No Fireplace
66,FireplaceQu,Gd,Good - Masonry Fireplace in main level
67,FireplaceQu,Fa,Fair - Prefabricated Fireplace in basement
68,FireplaceQu,TA,Average - Prefabricated Fireplace in main living area or
69,Fireplaces,NaN,NaN


As we can see, this missings just mean "no". Let's fill it and go further.

In [33]:
df['FireplaceQu'] = df['FireplaceQu'].cat.add_categories('none').fillna('none')

In [34]:
dicts('Lot')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
LotArea,Lot size in square feet,1951,0.00,9600,1.51,int64,cont,square feet
LotConfig,Lot configuration,5,0.00,Inside,73.07,object,nominal,-
LotFrontage,Linear feet of street connected to property,129,16.65,60,9.46,float64,cont,linear feet
LotShape,General shape of property,4,0.00,Reg,63.69,object,nominal,-


Value Description:


,Feature,Val,Value
110,LotArea,NaN,NaN
111,LotConfig,Corner,Corner lot
112,LotConfig,FR2,Frontage on 2 sides of property
113,LotConfig,CulDSac,Cul-de-sac
114,LotConfig,Inside,Inside lot
115,LotConfig,FR3,Frontage on 3 sides of property
116,LotFrontage,NaN,NaN
117,LotShape,Reg,Regular
118,LotShape,IR1,Slightly irregular
119,LotShape,IR2,Moderately Irregular


For a more correct filling, let's do it by grouping neighbourhoods.

In [35]:
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

To understand garage's characteristics we'll need to dive even more deeper.

In [36]:
dicts('Garage')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
GarageArea,Size of garage in square feet,604,0.03,0,5.38,float64,cont,square feet
GarageCars,Size of garage in car capacity,7,0.03,2,54.61,float64,discrete,quantity
GarageFinish,Interior finish of the garage,4,5.45,Unf,42.14,object,nominal,-
GarageType,Garage location,7,5.38,Attchd,59.03,object,nominal,-
GarageYrBlt,Year garage was built,104,5.45,2005,4.86,float64,time,year


Value Description:


,Feature,Val,Value
77,GarageArea,NaN,NaN
78,GarageCars,NaN,NaN
79,GarageFinish,RFn,Rough Finished
80,GarageFinish,Fin,Finished
81,GarageFinish,NaN,No Garage
82,GarageFinish,Unf,Unfinished
83,GarageType,NaN,No Garage
84,GarageType,CarPort,Car Port
85,GarageType,Basment,Basement Garage
86,GarageType,Attchd,Attached to home


In [37]:
for col in ['GarageFinish', 'GarageType']:
    df[col] = df[col].cat.add_categories('none').fillna('none')

In [38]:
gar = ddict[ddict.index.str.contains('Garage')].index

In [39]:
df[gar].loc[df['GarageArea'].isnull()]

,GarageArea,GarageCars,GarageFinish,GarageType,GarageYrBlt
2576,nan,nan,none,Detchd,nan


In [40]:
df[gar].loc[df['GarageCars'].isnull()]

,GarageArea,GarageCars,GarageFinish,GarageType,GarageYrBlt
2576,nan,nan,none,Detchd,nan


In [41]:
for col in ['GarageArea', 'GarageCars']:
    df[col] = df[col].fillna(0)

For 'GarageYrBlt' I'll check the value of 'GarageFinish' to be shure what it's no garage here:

In [42]:
tdf = df[gar].loc[df['GarageYrBlt'].isnull()]
tdf['GarageFinish'].unique()

[none]
Categories (1, object): [none]

In [43]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

In [44]:
df[gar].isnull().sum()

GarageArea      0
GarageCars      0
GarageFinish    0
GarageType      0
GarageYrBlt     0
dtype: int64

What the basement features are preparing for us?

In [45]:
dicts('Bsmt')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
BsmtExposure,Refers to walkout or garden level walls,5,2.81,No,65.23,object,ordinal,rate
BsmtFinSF1,Type 1 finished square feet,992,0.03,0,31.83,float64,cont,square feet
BsmtFinType1,Rating of basement finished area,7,2.71,Unf,29.15,object,ordinal,rate
BsmtFullBath,Basement full bathrooms,5,0.07,0,58.41,float64,discrete,quantity
BsmtQual,Evaluates the height of the basement,5,2.77,TA,43.95,object,ordinal,rate
BsmtUnfSF,Unfinished square feet of basement area,1136,0.03,0,8.26,float64,cont,square feet


Value Description:


,Feature,Val,Value
3,BsmtExposure,Gd,Good Exposure
4,BsmtExposure,Av,Average Exposure (split levels or foyers typically score
5,BsmtExposure,No,No Exposure
6,BsmtExposure,NaN,No Basement
7,BsmtExposure,Mn,Mimimum Exposure
8,BsmtFinSF1,NaN,NaN
9,BsmtFinType1,ALQ,Average Living Quarters
10,BsmtFinType1,NaN,No Basement
11,BsmtFinType1,Rec,Average Rec Room
12,BsmtFinType1,Unf,Unfinshed


In [46]:
noBsmt = ['BsmtQual', 'BsmtFinType1', 'BsmtExposure']
for col in noBsmt:
    df[col] = df[col].cat.add_categories('no basement').fillna('no basement')

In [47]:
df['BsmtExposure'] = df['BsmtExposure'].replace({'No':'no exposure'})

In [48]:
for col in ['BsmtFullBath', 'BsmtUnfSF', 'BsmtFinSF1']:
    df[col] = df[col].fillna(0)

In [49]:
df[noBsmt].isnull().sum()

BsmtQual        0
BsmtFinType1    0
BsmtExposure    0
dtype: int64

Massonry

In [50]:
dicts('Mas')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
MasVnrArea,Masonry veneer area in square feet,445,0.79,0,59.54,float64,cont,square feet
MasVnrType,Masonry veneer type,5,0.82,None,59.68,object,nominal,-


Value Description:


,Feature,Val,Value
145,MasVnrArea,NaN,NaN
146,MasVnrType,Stone,Stone
147,MasVnrType,BrkFace,Brick Face
148,MasVnrType,BrkCmn,Brick Common
149,MasVnrType,CBlock,Cinder Block
150,MasVnrType,None,None


In [51]:
df['MasVnrType'] = df['MasVnrType'].cat.add_categories('none').fillna('none')

In [52]:
tdf = df[['MasVnrType', 'MasVnrArea']].loc[df['MasVnrArea'].isnull()]
tdf['MasVnrType'].unique()

[none]
Categories (1, object): [none]

In [53]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [54]:
dicts('MSZ')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
MSZoning,Identifies the general zoning classification of the sale.,6,0.14,RL,77.60,object,nominal,-


Value Description:


,Feature,Val,Value
137,MSZoning,A,Agriculture
138,MSZoning,C,Commercial
139,MSZoning,FV,Floating Village Residential
140,MSZoning,I,Industrial
141,MSZoning,RH,Residential High Density
142,MSZoning,RL,Residential Low Density
143,MSZoning,RP,Residential Low Density Park
144,MSZoning,RM,Residential Medium Density


'MSZoning' identifies the general zoning classification of the sale - let's take a look to sales when MSZoning is missing

In [55]:
sale = ddict[ddict.index.str.contains('Sale')].index
sale = np.append(sale, ['MSZoning'])
df[sale].loc[df['MSZoning'].isnull()]

,SalePrice,MSZoning
1915,nan,NaN
2216,nan,NaN
2250,nan,NaN
2904,nan,NaN


All missing values in 'SalePrice' are from a test data set, so I'll fill with the top value 

In [56]:
df['MSZoning'] = df['MSZoning'].fillna('RL')

In [57]:
dicts('Exter')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
ExterQual,Evaluates the quality of the material on the exterior,4,0.00,TA,61.60,object,ordinal,rate
Exterior1st,Exterior covering on house,16,0.03,VinylSd,35.11,object,nominal,-
Exterior2nd,Exterior covering on house (if more than one material),17,0.03,VinylSd,34.74,object,nominal,-


Value Description:


,Feature,Val,Value
24,ExterQual,Po,Poor
25,ExterQual,Fa,Fair
26,ExterQual,TA,Average/Typical
27,ExterQual,Gd,Good
28,ExterQual,Ex,Excellent
29,Exterior1st,AsphShn,Asphalt Shingles
30,Exterior1st,AsbShng,Asbestos Shingles
31,Exterior1st,WdShing,Wood Shingles
32,Exterior1st,Stucco,Stucco
33,Exterior1st,BrkComm,Brick Common


In [58]:
ext = ddict[ddict.index.str.contains('Ext')].index
df[ext].loc[df['Exterior1st'].isnull()]

,ExterQual,Exterior1st,Exterior2nd
2151,TA,NaN,NaN


In [59]:
for col in ['Exterior1st', 'Exterior2nd']:
    df[col] = df[col].fillna('VinylSd')

In [60]:
dicts('Kitch')

Data Dictionary:


,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
KitchenQual,Kitchen quality,5,0.03,TA,51.11,object,ordinal,rate


Value Description:


,Feature,Val,Value
105,KitchenQual,Ex,Excellent
106,KitchenQual,TA,Typical/Average
107,KitchenQual,Fa,Fair
108,KitchenQual,Po,Poor
109,KitchenQual,Gd,Good


In [61]:
df['KitchenQual'] = df['KitchenQual'].fillna('TA')

I believe, work was just done, more accurate than it may be without using a Data Dictionary.
But let's do a final check of missings.

In [62]:
tdf = df.isnull().sum().sort_values(ascending=False)
tdf.loc[(tdf != 0)]

SalePrice    1459
dtype: int64

Good!

# 5. Manage Category

In [63]:
cols = df.select_dtypes('category').columns.to_list()
ddict.loc[ddict.index.isin(cols)]

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
BsmtFinType1,Rating of basement finished area,7,2.71,Unf,29.15,object,ordinal,rate
BsmtQual,Evaluates the height of the basement,5,2.77,TA,43.95,object,ordinal,rate
ExterQual,Evaluates the quality of the material on the exterior,4,0.00,TA,61.60,object,ordinal,rate
Exterior1st,Exterior covering on house,16,0.03,VinylSd,35.11,object,nominal,-
Exterior2nd,Exterior covering on house (if more than one material),17,0.03,VinylSd,34.74,object,nominal,-
FireplaceQu,Fireplace quality,6,48.65,Gd,25.49,object,ordinal,rate
Foundation,Type of foundation,6,0.00,PConc,44.81,object,nominal,-
GarageFinish,Interior finish of the garage,4,5.45,Unf,42.14,object,nominal,-
GarageType,Garage location,7,5.38,Attchd,59.03,object,nominal,-
HeatingQC,Heating quality and condition,5,0.00,Ex,51.15,object,ordinal,rate


Filtering the 'quality' columns:

In [64]:
cols = ['OverallCond', 'OverallQual', 'FireplaceQu', 'KitchenQual', 'ExterQual', 'BsmtQual', 'HeatingQC']
for col in cols:
    print(col, ':', df[col].unique())

OverallCond : [5 8 6 7 4 2 3 9 1]
OverallQual : [ 7  6  8  5  9  4 10  3  1  2]
FireplaceQu : [none, TA, Gd, Fa, Ex, Po]
Categories (6, object): [none, TA, Gd, Fa, Ex, Po]
KitchenQual : [Gd, TA, Ex, Fa]
Categories (4, object): [Gd, TA, Ex, Fa]
ExterQual : [Gd, TA, Ex, Fa]
Categories (4, object): [Gd, TA, Ex, Fa]
BsmtQual : [Gd, TA, Ex, no basement, Fa]
Categories (5, object): [Gd, TA, Ex, no basement, Fa]
HeatingQC : [Ex, Gd, TA, Fa, Po]
Categories (5, object): [Ex, Gd, TA, Fa, Po]


Renaming values:

In [65]:
for col in ['OverallCond', 'OverallQual']:
    df[col] = df[col].replace({1:'Very Poor', 2:'Poor', 3:'Fair', 4:'Below Average', 5:'Average',
                               6:'Above Average', 7:'Good', 8:'Very Good', 9:'Excellent', 
                               10:'Very Excellent'}).astype('category')

In [66]:
for col in ['FireplaceQu', 'KitchenQual', 'ExterQual', 'BsmtQual', 'HeatingQC']:
    df[col] = df[col].replace({'TA':'Typical', 'Gd':'Good', 'Ex':'Excellent', 'Po':'Poor', 
                               'Fa':'Fair'}).astype('category')

# 6. Dates

In [67]:
tdf = ddict.loc[ddict['Type'].isin(['time'])]
tdf

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
GarageYrBlt,Year garage was built,104,5.45,2005,4.86,float64,time,year
MoSold,Month Sold (MM),12,0.00,6,17.23,int64,time,month
YearBuilt,Original construction date,118,0.00,2005,4.86,int64,time,year
YearRemodAdd,Remodel date (same as construction date if no remodel,61,0.00,1950,12.37,int64,time,year
YrSold,Year Sold (YYYY),5,0.00,2007,23.71,int64,time,year


In [68]:
df[tdf.index.unique().tolist()].head(2)

,GarageYrBlt,MoSold,YearBuilt,YearRemodAdd,YrSold
0,"2,003.00",2,2003,2003,2008
1,"1,976.00",5,1976,1976,2007


In [69]:
df['GarageYrBlt'] = df['GarageYrBlt'].astype('str').map(lambda x: x.split('.')[0])

In [70]:
from datetime import date
df['YearMoSold'] = pd.to_datetime(df['YrSold'].astype('str')+'-'+  df['MoSold'].astype('str'))
df = df.drop(['MoSold', 'YrSold'], axis=1)
df['YearMoSold'].head(2)

0   2008-02-01
1   2007-05-01
Name: YearMoSold, dtype: datetime64[ns]

For the future using I'll change type as string because when fill will open next time it will be readed as object. But to show you the importanse for memory usage, I'll change type to category here

In [71]:
for col in ['YearRemodAdd', 'YearBuilt']:
    df[col] = df[col].astype('str')

In [72]:
for col in ['YearRemodAdd', 'YearBuilt', 'GarageYrBlt']:
    df[col] = df[col].astype('category')

# 7. Final check

In [73]:
# missing values
df.isnull().sum().nlargest().to_frame()

,0
SalePrice,1459
MSSubClass,0
MSZoning,0
LotFrontage,0
LotArea,0


In [74]:
# data types
df.get_dtype_counts().to_frame().T

,category,datetime64[ns],float64,int64,object
0,23,1,8,11,1


In [75]:
df.dtypes[df.dtypes == 'object']

BsmtExposure    object
dtype: object

In [76]:
df['BsmtExposure'] = df['BsmtExposure'].astype('category')

In [77]:
# memory usage detailed
df.memory_usage(deep=True).nlargest().to_frame()

,0
MSSubClass,23352
LotFrontage,23352
LotArea,23352
MasVnrArea,23352
BsmtFinSF1,23352


In [78]:
# memory usage in common
mu_after = round(df.memory_usage(index=True).sum()/1024, 2)
print('Now the data set uses', mu_after, 'of memory against', mu_before, '(MB)')

Now the data set uses 546.07 of memory against 1824.5 (MB)


So huge difference!

# 8. Saving results

In [79]:
#old train
print('last index:', train.index[-1], 'shape:', train.shape)

last index: 1460 shape: (1460, 80)


In [80]:
# new train
new_train = df.iloc[:1460,:]
new_train.shape

(1460, 44)

In [81]:
# simple check
new_train['SalePrice'].isnull().sum()

0

Correct!

In [82]:
# new test
new_test = df.iloc[1460:,:]
new_test.shape

(1459, 44)

In [83]:
# the same chek again
new_test['SalePrice'].unique()

array([nan])

#### Change the data dictionary for a further using:

In [84]:
# deleting columns
ddict = ddict.drop(['%Missing'], axis=1)

In [85]:
# deleting rows
todelete=['MoSold', 'YrSold']
ddict = ddict.loc[~ddict.index.isin(todelete)]

In [86]:
# check if any mistakes
set1 = set(df.columns.tolist())
set2 = set(ddict.index.tolist())
set1.difference(set2)

{'YearMoSold'}

We need to add a created feature into a dictionary

In [87]:
# combining a row
n = df['YearMoSold'].nunique()
values = df['YearMoSold'].value_counts().to_frame().reset_index()
t = values.iloc[0,0]
u = values.iloc[0,1]*100/len(df)
dt = str(df['YearMoSold'].dtype)
a = pd.DataFrame({'Description':'date', '#Unique':n, 'TopValue':t, '%UsedTop':u, 'Dtype':dt, 'Type':'time', 'Unit':'time'}, index=['YearMoSold'])

In [88]:
# adding the row
ddict = ddict.append(a)

In [89]:
# Saving a new data types as a column in ddict
for i in ddict.index:
    ddict.loc[i,'Dtype'] = str(df[i].dtype)
ddict.head()

,Description,#Unique,TopValue,%UsedTop,Dtype,Type,Unit
1stFlrSF,First Floor square feet,1083,864,1.58,int64,cont,square feet
2ndFlrSF,Second floor square feet,635,0,57.14,int64,cont,square feet
BedroomAbvGr,Bedrooms above grade (does NOT include basement bedrooms),8,3,54.68,int64,discrete,quantity
BsmtExposure,Refers to walkout or garden level walls,5,No,65.23,category,ordinal,rate
BsmtFinSF1,Type 1 finished square feet,992,0,31.83,float64,cont,square feet


In [90]:
# saving outputs
new_train.to_csv('.../HPrices_train_cleaned.csv', index=False)
new_test.to_csv('.../HPrices_test_cleaned.csv', index=False)
ddict.to_csv('.../HPrices_ddict_cleaned.csv')

That's it!  
I've done here a data cleaning only, because I prefer to work with milestones.  